In [4]:
import torch
from model.cnn import CNN
from settings import *
import torch.onnx as onnx

In [5]:
# Load checkpoint
checkpoint = torch.load('models/trained_model_63.96.pt')
settings = checkpoint['model_settings']
cnn_settings = CNNSettings(**settings)
model = CNN(**settings)
model.load_state_dict(checkpoint['model'])
model

CNN(
  (activation): SELU()
  (conv1): ResidualBlock(
    (conv1): CausalConv2d(
      (conv): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 2), bias=False)
    )
    (conv2): CausalConv2d(
      (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 2), bias=False)
    )
    (norm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (norm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): SELU()
    (re_sample): Conv2d(1, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
  (pool1): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  (conv2): ResidualBlock(
    (conv1): CausalConv2d(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 2), bias=False)
    )
    (conv2): CausalConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 2), bias=False)
    )
    (norm1): BatchNorm2d(32, eps

In [8]:
export_options = torch.onnx.ExportOptions(dynamic_shapes=True)
onnx.export(
    model, 
    torch.zeros(1, cnn_settings.n_mels, 12),
    "./models/cnn.onnx",
    input_names=["mel_spec"],
    output_names=["activation"],
    dynamic_axes={
        "mel_spec": [0, 2],
        "activation": [0, 2],
    },
)